In [1]:
!pip install transformer_lens
!pip install einops
!pip install jaxtyping

In [1]:
import datasets

import torch as t
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from utils import Config, TransformerTrainingArgs
from transformer_modules import DemoTransformer
from transformers import GPT2Tokenizer
from trainer import TransformerTrainer

device = t.device('mps' if t.backends.mps.is_available() else 'cuda' if t.cuda.is_available() else 'cpu')

%load_ext autoreload
%autoreload 2

In [7]:
model_cfg = Config(
    debug=False,
    apply_UmuP=True,
    d_model=256,
    n_heads=4,
    d_head=64,
    d_mlp=1024,
    n_layers=2,
    n_ctx=256,
    d_vocab=50257 ## GPT2 vocab size
)



In [10]:
args = TransformerTrainingArgs(
    lr=1e-1,
    weight_decay=1e-2,
    epochs=20,
    max_steps_per_epoch=100,
    batch_size=16,
    wandb_project="mup-transformer",
    wandb_name="transformer-test-mup"
)

In [4]:
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train").remove_columns("meta")
print(dataset)
print(dataset[0]['text'][:100])

Dataset({
    features: ['text'],
    num_rows: 10000
})
It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playi


In [5]:
from transformer_lens.utils import tokenize_and_concatenate

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenized_dataset = tokenize_and_concatenate(dataset, tokenizer, streaming=False, max_length=model_cfg.n_ctx, column_name="text", add_bos_token=True, num_proc=4)

dataset_dict = tokenized_dataset.train_test_split(test_size=1000)

In [11]:
model = DemoTransformer(model_cfg).to(device)
trainer = TransformerTrainer(args, model, dataset_dict)
trainer.train()

Epoch 20, loss: 9.171, accuracy: 0.031: : 2020it [03:32, 12.13it/s]                        

accuracy,███▇▁█▃▁▁▁▇█████████
train_loss,█████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▂▁
accuracy,0.03122
train_loss,9.17105


/opt/conda/envs/default/lib/python3.11/site-packages/wandb/sdk/wandb_run.py:2386: UserWarning: Run (jrt8h6op) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
Epoch 20, loss: 9.171, accuracy: 0.031: : 2020it [03:37,  9.31it/s]
